# Schrödinger 配体对接

Schrödinger 配体对接是指利用 Schrödinger 软件对接蛋白质和配体的过程。本教程将介绍配体对接的基本步骤，并提供相应的 Python 代码。

首先，我们需要安装 Schrödinger 软件并激活环境。具体安装方法请参考 Schrödinger 官方文档。

激活环境后，我们需要准备以下文件：
- 蛋白质 PDB 文件
- 配体 SMILES 字符串

接下来，我们将依次介绍配体对接的五个步骤：

1. **准备蛋白质**：通过 Schrödinger 的 Protein Preparation Wizard 处理蛋白质。
2. **自动计算网格中心**：基于 PDB 文件的配体残基位置来自动计算网格中心。
3. **配体处理**：将 SMILES 转换为 SDF 格式的配体文件。
4. **生成网格**：根据计算出的网格中心，生成用于对接的网格文件。
5. **执行分子对接**：使用 Glide 进行分子对接。

安装完成并正确激活环境后，你可以通过以下方式运行 Python 脚本：
$SCHRODINGER/run python Schrodinger_docking.py

### 准备蛋白质步骤 1：蛋白质准备

In [ ]:
from schrodinger.protein import prepwizard

def prepare_protein(pdb_file, output_mae_file):
    """
    使用 Schrödinger Protein Preparation Wizard 处理蛋白质文件
    """
    # Initialize the Protein Preparation Wizard
    prep_wizard = prepwizard.ProteinWizard(pdb_file)
    
    # Perform standard steps: Assign bond orders, add hydrogens, etc.
    prep_wizard.process()
    
    # Save the prepared protein structure to a .mae file
    prep_wizard.write_structure(output_mae_file)
    print(f"Protein saved to {output_mae_file}")

# 调用函数准备蛋白质
pdb_file = "protein.pdb"  # 你的蛋白质PDB文件路径
protein_prep_path = "protein_prepped.mae"
prepare_protein(pdb_file, protein_prep_path)


## 步骤 2：自动计算网格中心
我们将从 PDB 文件中提取配体的残基位置，然后基于这些原子的几何中心计算网格中心。

In [ ]:
from schrodinger.structure import StructureReader
import numpy as np

def detect_ligand_resname(structure):
    """
    自动检测配体残基名称。
    假设配体是蛋白质以外的其他分子。
    
    Parameters
    ----------
    structure: schrodinger.structure.Structure
        从PDB读取的结构对象。
    
    Returns
    -------
    ligand_resname: str
        自动检测到的配体残基名称。
    """
    ligand_resname = None
    for atom in structure.atom:
        # 检查是否为非蛋白质残基
        if not atom.is_protein:
            ligand_resname = atom.pdbres
            break  # 找到一个配体残基后可以停止
    return ligand_resname

def calculate_grid_center(pdb_file):
    """
    从PDB文件中提取配体残基的几何中心，用作对接网格的中心坐标。
    
    Parameters
    ----------
    pdb_file: str
        蛋白质PDB文件路径
    
    Returns
    -------
    grid_center: tuple
        计算得到的网格中心坐标 (x, y, z)
    ligand_resname: str
        自动检测到的配体残基名称
    """
    with StructureReader(pdb_file) as structures:
        for structure in structures:
            # 自动检测配体残基名称
            ligand_resname = detect_ligand_resname(structure)
            
            if ligand_resname is None:
                print("未能检测到配体残基名称")
                return None, None
            
            print(f"Detected ligand residue name: {ligand_resname}")
            
            # 筛选出指定配体残基的原子
            ligand_atoms = [atom for atom in structure.atom if atom.pdbres == ligand_resname]
            
            if not ligand_atoms:
                print(f"配体 {ligand_resname} 中没有找到原子")
                return None, None
            
            # 提取原子坐标
            coords = np.array([atom.xyz for atom in ligand_atoms])
            
            # 计算几何中心
            grid_center = np.mean(coords, axis=0)
            return tuple(grid_center), ligand_resname

# 调用函数计算网格中心
grid_center, ligand_resname = calculate_grid_center(pdb_file)

if grid_center:
    print(f"Calculated grid center: {grid_center}")
    print(f"Detected ligand residue name: {ligand_resname}")


## 步骤 3：配体处理
我们将使用 SMILES 格式的配体描述符来生成配体文件。

In [ ]:
from schrodinger.structutils.build import smiles_to_structure
from schrodinger.structure import StructureWriter

def convert_smiles_to_sdf(smiles, output_sdf_file):
    """
    将 SMILES 字符串转换为 SDF 格式并保存
    """
    # Convert SMILES to 3D structure
    lig_structure = smiles_to_structure(smiles)
    
    # Save the structure to an SDF file
    with StructureWriter(output_sdf_file) as writer:
        writer.append(lig_structure)
    print(f"Ligand saved to {output_sdf_file}")

# 调用函数处理配体
smiles = "CCO"  # 示例的 SMILES 字符串
ligand_file_path = "ligands.sdf"
convert_smiles_to_sdf(smiles, ligand_file_path)


## 步骤 4：生成网格
我们将使用网格中心和配体文件来生成用于对接的网格文件。

In [ ]:
from schrodinger.application.glide import Grid

def generate_docking_grid(protein_prep_path, grid_center, output_grid_file):
    """
    为对接生成网格文件
    """
    grid_gen = Grid()
    grid_gen.input_structure = protein_prep_path
    grid_gen.grid_center = grid_center  # 设置网格中心
    grid_gen.output_file = output_grid_file
    grid_gen.run()  # 生成网格文件
    print(f"Grid saved to {output_grid_file}")

# 调用函数生成网格
grid_file_path = "grid.zip"
generate_docking_grid(protein_prep_path, grid_center, grid_file_path)


## 步骤 5：执行分子对接
我们将使用 Glide 软件来执行分子对接。

In [ ]:
from schrodinger.application.glide import GlideJob

def run_glide_docking(grid_file_path, ligand_file_path, docking_output_file):
    """
    使用 Glide 进行分子对接
    """
    glide_job = GlideJob()
    glide_job.grid = grid_file_path  # 使用生成的网格
    glide_job.add_ligands(ligand_file_path)  # 添加配体
    glide_job.output_file = docking_output_file
    glide_job.run()  # 运行对接
    print(f"Docking results saved to {docking_output_file}")

# 调用函数执行对接
docking_output = "docking_output.maegz"
run_glide_docking(grid_file_path, ligand_file_path, docking_output)
